## Import using package

In [1]:
import cv2
import time
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm as tqdm

In [12]:
def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

def point_in_box(point_boxs_list,check_point):
    point_pair_list = [[point_boxs_list[i-1],point_boxs_list[i]] for i in range(len(point_boxs_list))]
    box_area = PolygonArea(point_boxs_list)
    count_area = 0
    for edge in point_pair_list:
        edge.append(check_point)
        count_area = PolygonArea(edge)+count_area
    if count_area - box_area < 1:
        return True
    else:
        return False
    
def show_pixel_set(img_nparray):
    a = img_nparray
    unique, counts = np.unique(a, return_counts=True)
    return dict(zip(unique, counts))

def get_check_point(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(np.where(img == 255,255.,0.).astype('uint8'), cv2.COLOR_BGR2GRAY)
    img = np.where(img == 76,255,0)
    image_name = img_path.split('/')[-1][:-7]
#     if image_name in 'NN_160212_152938_9C3C6C NN_160217_094616_16D99F NN_161012_105932_C0383A NN_161012_110001_C03833 NN_170328_100636_9CADE3 NN_190923_111212_4B148':
#         print(img_path)
#         plt.imshow(img)
#         plt.show()
    return (np.mean(np.nonzero(img)[1]),np.mean(np.nonzero(img)[0]))

#print_ = False
def get_DM(num,side):
    '''
    tooth_num:
    if tooth_num <= 8 and tooth_num >= 25:
        L = D  
        R = M
    else:
        L = M
        R = D        
    '''
    if ((int(num) <= 8 or int(num) >= 25) and side =='L') or (25 > int(num) > 8 and side =='R'):
        return '{}D'.format(num)
    else:
        return '{}M'.format(num)
    
def replac_value(image_name,M_tooth_info_b,D_info):
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_predict_tooth'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_value'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'PBL_stage'] = '{}'.format('No_match')
    D_info.loc[(D_info['image_name'] == '{}'.format(image_name)),'tooth_path'] = '{}'.format('No_match')
    for tooth in M_tooth_info_b['match_tooth']:
        if tooth != 'No_match':
            tooth_num = tooth.split('_')[0]
            tooth_side = tooth.split('_')[1]
            tooth_left = get_DM(tooth_num,'L') if 'L' in tooth_side else None
            tooth_right = get_DM(tooth_num,'R') if 'R' in tooth_side else None

            LPBL = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['LPBL(%)'].values[0]
            L_Stage = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['L_Stage(int)']
            RPBL = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['RPBL(%)'].values[0]
            R_Stage = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['R_Stage(int)']
            tooth_path = M_tooth_info_b.loc[M_tooth_info_b['match_tooth'] == '{}'.format(tooth)]['Tooth_num'].values[0]
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_left)), 'PBL_predict_tooth'] = '{}'.format(tooth_num+'_L') 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_right)), 'PBL_predict_tooth'] = '{}'.format(tooth_num+'_R')
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_left)), 'PBL_value'] = '{}'.format(LPBL,'.3f') 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_right)), 'PBL_value'] = '{}'.format(RPBL,'.3f')
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_left)), 'PBL_stage'] = '{}'.format(L_Stage.values[0])
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_right)), 'PBL_stage'] = '{}'.format(R_Stage.values[0])
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_left)), 'tooth_path'] = '{}'.format(tooth_path) 
            D_info.loc[(D_info['image_name'] == '{}'.format(image_name)) & (D_info['tooth_num'] == '{}'.format(tooth_right)), 'tooth_path'] = '{}'.format(tooth_path) 
            if image_name in 'NN_160212_152938_9C3C6C NN_160217_094616_16D99F NN_161012_105932_C0383A NN_161012_110001_C03833 NN_170328_100636_9CADE3 NN_190923_111212_4B148':
                print(image_name)
                print(tooth_side)
                print('tooth_left',tooth_left)
                print('tooth_right',tooth_right)
                print('LPBL:{}, L_Stage:{},tooth_path:{}'.format(LPBL,L_Stage,tooth_path))
                print('RPBL:{}, R_Stage:{},tooth_path:{}'.format(RPBL,R_Stage,tooth_path))
                print('--------------------------')
                
    return D_info


## Load model_detection csv & dentist_prediction 

In [13]:
Model_detection_result_csv_path = '/root/notebooks/tooth_project/tooth_xray_segmention/PBL_detection/PBL_result.csv'
Dentist_predict_result_csv_path = '/root/notebooks/tooth_project/tooth_xray_segmention/PBL_detection/csv_file/PBL_Table_2020_08_Clean.csv'


In [14]:
Model_detection = pd.read_csv(Model_detection_result_csv_path)
Dentist_predict = pd.read_csv(Dentist_predict_result_csv_path)

## Show info check non_Null

In [15]:
Model_detection.info()
print('------------------------------------------------')
Dentist_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 7 columns):
Image_name             946 non-null object
Tooth_num              946 non-null object
LPBL(%)                946 non-null float64
L_Stage(int)           946 non-null int64
RPBL(%)                946 non-null float64
R_Stage(int)           946 non-null int64
tooth_pbl_line_path    946 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 51.8+ KB
------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23019 entries, 0 to 23018
Data columns (total 4 columns):
sub_dir_name    23019 non-null object
image_name      23019 non-null object
tooth_num       23019 non-null object
state           23019 non-null int64
dtypes: int64(1), object(3)
memory usage: 719.4+ KB


## Show the first three data

In [16]:
Model_detection.head(3)

,Image_name,Tooth_num,LPBL(%),L_Stage(int),RPBL(%),R_Stage(int),tooth_pbl_line_path
0,NN_090903_154047_C07485,NN_090903_154047_C07485_1.PNG,0.0,0,4.948,1,/root/notebooks/tooth_project/tooth_xray_segme...
1,NN_090903_154047_C07485,NN_090903_154047_C07485_2.PNG,-999.0,-999,0.000,0,/root/notebooks/tooth_project/tooth_xray_segme...
2,NN_090903_154047_C07485,NN_090903_154047_C07485_3.PNG,0.0,0,16.544,2,/root/notebooks/tooth_project/tooth_xray_segme...


In [17]:
Dentist_predict.head(3)

,sub_dir_name,image_name,tooth_num,state
0,701509 061919 x,NN_190619_104716_EA3E9,8M,-999
1,701509 061919 x,NN_190619_104716_EA3E9,9M,1
2,701509 061919 x,NN_190619_104716_EA3E9,9D,1


In [18]:
D_tooth_info = Dentist_predict
D_tooth_info["PBL_predict_tooth"] = np.nan
D_tooth_info["PBL_value"] = np.nan
D_tooth_info["PBL_stage"] = np.nan
D_tooth_info["tooth_path"] = np.nan
D_tooth_info.head(3)

,sub_dir_name,image_name,tooth_num,state,PBL_predict_tooth,PBL_value,PBL_stage,tooth_path
0,701509 061919 x,NN_190619_104716_EA3E9,8M,-999,NaN,NaN,NaN,NaN
1,701509 061919 x,NN_190619_104716_EA3E9,9M,1,NaN,NaN,NaN,NaN
2,701509 061919 x,NN_190619_104716_EA3E9,9D,1,NaN,NaN,NaN,NaN


## Check comparsion image name

In [19]:
list_image_name = list(set(list(Model_detection['Image_name'])))
list_image_name.sort()

json_save_path = '/root/notebooks/tooth_project/tooth_xray_segmention/Data/patient_folder/*/*/*.json'
json_file_list = glob.glob(json_save_path)

show = True
json_ = True

count = 0
for image_name in tqdm(list_image_name ,total=len(list_image_name)):
    match_tooth = []
    M_tooth_info = Model_detection.loc[(Model_detection['Image_name'] == image_name)]
    if True:
        for index, img_path in enumerate(M_tooth_info[['tooth_pbl_line_path'][0]]):
            check_in_box = False
            True_tooth_label = "No_match"
            if json_:
                check_point = get_check_point(img_path)
                json_file_path = [json_name for json_name in json_file_list if image_name in json_name]
                if json_file_path != []:
                    with open(json_file_path[0], 'r') as file:
                        json_file = json.load(file)
                    for tooth in json_file['shapes']:                
                        check_in_box = point_in_box(tooth['points'],check_point)
                        if check_in_box:   
                            True_tooth_label = tooth['label']
                            break
            count += 1
            match_tooth.append(True_tooth_label)
    tooth = match_tooth
    M_tooth_info['match_tooth'] = tooth
    if image_name in 'NN_160212_152938_9C3C6C NN_160217_094616_16D99F NN_161012_105932_C0383A NN_161012_110001_C03833 NN_170328_100636_9CADE3 NN_190923_111212_4B148':
        print(M_tooth_info)
    D_tooth_info = replac_value(image_name,M_tooth_info,D_tooth_info)
print(count)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                  Image_name                      Tooth_num  LPBL(%)  \
138  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_1.PNG -999.000   
139  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_2.PNG   15.686   
140  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_3.PNG    6.044   
141  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_4.PNG   19.292   
142  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_5.PNG -999.000   
611  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_1.PNG -999.000   
612  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_2.PNG   15.686   
613  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_3.PNG    6.044   
614  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_4.PNG   19.292   
615  NN_160212_152938_9C3C6C  NN_160212_152938_9C3C6C_5.PNG -999.000   

     L_Stage(int)  RPBL(%)  R_Stage(int)  \
138          -999    0.000             0   
139             2 -999.000          -999   
140             1 -999.000          -999   
141             2   24.907     

In [20]:
D_tooth_info_b = D_tooth_info[['image_name','tooth_num','state','PBL_predict_tooth','PBL_value','PBL_stage','tooth_path']]
M_tooth_info_b = M_tooth_info[['Image_name','LPBL(%)','L_Stage(int)','RPBL(%)','R_Stage(int)']]

In [21]:
D_tooth_info_b = pd.DataFrame(D_tooth_info_b)
D_tooth_info_b.to_csv('../csv_file/PBL_match_result.csv')